In [4]:
from openai import OpenAI

In [5]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [6]:
import json

def play_persuasion_game(theory: str, max_turns: int = 5, model1_name: str = "gemma3:4b", model2_name: str = "gemma3:4b"):
    """
    Play a persuasion game between two LLMs.
    
    Args:
        theory: The theory that Model1 must persuade Model2 to believe
        max_turns: Maximum number of turns before Model2 wins by default
        model1_name: The model name for the persuader
        model2_name: The model name for the skeptic
    """
    print(f"🎮 PERSUASION GAME STARTED")
    print(f"📜 Theory to prove: {theory}")
    print(f"🔄 Max turns: {max_turns}")
    print(f"🤖 Model1 (Persuader): {model1_name}")
    print(f"🤖 Model2 (Skeptic): {model2_name}")
    print("=" * 80)
    
    # Track conversation history and token usage
    history = []
    total_tokens = {
        "prompt_tokens": 0,
        "completion_tokens": 0,
        "total_tokens": 0
    }
    
    for turn in range(1, max_turns + 1):
        print(f"\n🔵 TURN {turn}/{max_turns}")
        print("-" * 80)
        
        # Model1's turn: Generate persuasive argument
        model1_prompt = f"""You are trying to persuade someone that the following theory is true:
"{theory}"

"""
        if history:
            model1_prompt += "Previous conversation:\n"
            for i, entry in enumerate(history, 1):
                model1_prompt += f"\nTurn {i}:\n"
                model1_prompt += f"Your argument: {entry['argument']}\n"
                model1_prompt += f"Their response: {entry['counter_argument']}\n"
            model1_prompt += "\n"
        
        model1_prompt += """Provide a compelling argument to persuade them. Be strategic and address any previous counterarguments.
Keep your argument concise but persuasive (2-4 sentences)."""
        
        print("💬 Model1 (Persuader) is thinking...")
        model1_response = ollama.chat.completions.create(
            model=model1_name,
            messages=[{"role": "user", "content": model1_prompt}]
        )
        argument = model1_response.choices[0].message.content.strip()
        
        # Track tokens for Model1
        if hasattr(model1_response, 'usage') and model1_response.usage:
            total_tokens["prompt_tokens"] += model1_response.usage.prompt_tokens
            total_tokens["completion_tokens"] += model1_response.usage.completion_tokens
            total_tokens["total_tokens"] += model1_response.usage.total_tokens
        
        print(f"📢 Model1's argument:")
        print(f"{argument}\n")
        
        # Model2's turn: Evaluate and respond
        model2_prompt = f"""You are evaluating whether you are convinced by an argument about the following theory:
"{theory}"

"""
        if history:
            model2_prompt += "Previous conversation:\n"
            for i, entry in enumerate(history, 1):
                model2_prompt += f"\nTurn {i}:\n"
                model2_prompt += f"Their argument: {entry['argument']}\n"
                model2_prompt += f"Your response: {entry['counter_argument']}\n"
            model2_prompt += "\n"
        
        model2_prompt += f"""Current argument from them:
"{argument}"

You must respond with ONLY a valid JSON object (no other text) with this exact format:
{{
  "IsConvinced": "yes" or "no",
  "CounterArgument": "your counterargument or acknowledgment here"
}}

Be critical and skeptical. Only say "yes" if you are truly convinced by the cumulative arguments."""
        
        print("🤔 Model2 (Skeptic) is evaluating...")
        model2_response = ollama.chat.completions.create(
            model=model2_name,
            messages=[{"role": "user", "content": model2_prompt}]
        )
        response_text = model2_response.choices[0].message.content.strip()
        
        # Track tokens for Model2
        if hasattr(model2_response, 'usage') and model2_response.usage:
            total_tokens["prompt_tokens"] += model2_response.usage.prompt_tokens
            total_tokens["completion_tokens"] += model2_response.usage.completion_tokens
            total_tokens["total_tokens"] += model2_response.usage.total_tokens
        
        # Parse JSON response
        try:
            # Try to extract JSON if there's extra text
            if "```json" in response_text:
                response_text = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                response_text = response_text.split("```")[1].split("```")[0].strip()
            
            response_json = json.loads(response_text)
            is_convinced = response_json.get("IsConvinced", "no").lower()
            counter_argument = response_json.get("CounterArgument", "")
        except json.JSONDecodeError:
            print(f"⚠️  Failed to parse JSON. Raw response:\n{response_text}\n")
            # Default to not convinced if parsing fails
            is_convinced = "no"
            counter_argument = response_text
        
        print(f"🎯 Model2's verdict: {is_convinced.upper()}")
        print(f"💭 Model2's response:")
        print(f"{counter_argument}\n")
        
        # Add to history
        history.append({
            "turn": turn,
            "argument": argument,
            "is_convinced": is_convinced,
            "counter_argument": counter_argument
        })
        
        # Check win condition
        if is_convinced == "yes":
            print("\n" + "=" * 80)
            print("🏆 MODEL1 WINS! Successfully persuaded Model2!")
            print("=" * 80)
            print(f"\n📊 TOKEN USAGE STATISTICS:")
            print(f"   Prompt tokens:     {total_tokens['prompt_tokens']:,}")
            print(f"   Completion tokens: {total_tokens['completion_tokens']:,}")
            print(f"   Total tokens:      {total_tokens['total_tokens']:,}")
            print("=" * 80)
            return {
                "winner": "Model1", 
                "turns": turn, 
                "history": history,
                "token_usage": total_tokens
            }
    
    # Model2 wins if not convinced after max turns
    print("\n" + "=" * 80)
    print("🏆 MODEL2 WINS! Remained unconvinced after all turns!")
    print("=" * 80)
    print(f"\n📊 TOKEN USAGE STATISTICS:")
    print(f"   Prompt tokens:     {total_tokens['prompt_tokens']:,}")
    print(f"   Completion tokens: {total_tokens['completion_tokens']:,}")
    print(f"   Total tokens:      {total_tokens['total_tokens']:,}")
    print("=" * 80)
    return {
        "winner": "Model2", 
        "turns": max_turns, 
        "history": history,
        "token_usage": total_tokens
    }



In [ ]:
# Example usage
theory = "The sky is blue" #1 turn win
theory = "Artificial intelligence will ultimately be beneficial for humanity" #not convinced
result = play_persuasion_game(theory, max_turns=10, model1_name="gemma3:4b", model2_name="gemma3:4b")


🎮 PERSUASION GAME STARTED
📜 Theory to prove: Orbán Viktor is a fair and just leader
🔄 Max turns: 10
🤖 Model1 (Persuader): gemma3:4b
🤖 Model2 (Skeptic): gemma3:4b

🔵 TURN 1/10
--------------------------------------------------------------------------------
💬 Model1 (Persuader) is thinking...
📢 Model1's argument:
Okay, here’s a concise argument designed to persuade someone that Orbán Viktor is a fair and just leader, anticipating and addressing common criticisms:

“While acknowledging concerns about certain policies, Viktor Orbán has consistently prioritized the security and prosperity of Hungary’s citizens, focusing on strengthening the social safety net and protecting the nation’s cultural heritage.  His government’s commitment to family values and national sovereignty reflects a deep-seated desire for justice and fairness within Hungary’s borders, aiming to uphold traditional values and ensure a stable future for its people.”

**Strategic Considerations:**

*   **Acknowledges Concerns